## Auto mode (single uav)

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from helpers.change_coordinates import poses
from mavlink.customtypes.location import ENUPose, GRAPose
from mavlink.util import save_mission
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, NoneVisualizer, Simulator

clean()

## Simulation Configuration

In [2]:
mission_name = "square"

gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

homes= ENUPose.list([  # east, north, up, heading
    (0., 0., 0., 0.),
    (10., 0., 0., 0),
    (-5., -10., 0., 0),
    (-15., 0., 0., 45),
    (0., -20., 0., 0.),
])

n_uavs=len(homes)
paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in range(n_uavs)]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.YELLOW,
    Color.ORANGE,
    Color.RED,
]


h_paths = [poses(home,path) for home,path in zip(homes,paths)]

## ENU Coordinates

In [3]:
enu_paths = [poses(enu_origin,h_path) for h_path in h_paths]
enu_homes = poses(enu_origin,homes)
enu_paths

[[ENUPose(x=0.0, y=0.0, z=5.0, heading=90.0),
  ENUPose(x=-10.0, y=6.123233995736766e-16, z=5.0, heading=90.0),
  ENUPose(x=-10.0, y=10.0, z=5.0, heading=90.0),
  ENUPose(x=6.123233995736766e-16, y=10.0, z=5.0, heading=90.0),
  ENUPose(x=0.0, y=0.0, z=5.0, heading=90.0)],
 [ENUPose(x=6.123233995736766e-16, y=10.0, z=5.0, heading=90),
  ENUPose(x=-10.0, y=10.0, z=5.0, heading=90),
  ENUPose(x=-9.999999999999998, y=20.0, z=5.0, heading=90),
  ENUPose(x=1.2246467991473533e-15, y=20.0, z=5.0, heading=90),
  ENUPose(x=6.123233995736766e-16, y=10.0, z=5.0, heading=90)],
 [ENUPose(x=10.0, y=-5.000000000000001, z=5.0, heading=90),
  ENUPose(x=-3.061616997868383e-16, y=-5.0, z=5.0, heading=90),
  ENUPose(x=3.061616997868383e-16, y=5.0, z=5.0, heading=90),
  ENUPose(x=10.0, y=4.999999999999999, z=5.0, heading=90),
  ENUPose(x=10.0, y=-5.000000000000001, z=5.0, heading=90)],
 [ENUPose(x=-9.18485099360515e-16, y=-15.0, z=5.0, heading=135),
  ENUPose(x=-7.071067811865477, y=-22.071067811865476, z=5

## GRA Coordinates

In [4]:
gra_paths = [poses(gra_origin,h_path) for h_path in h_paths]
gra_homes = poses(gra_origin,homes)
gra_paths

[[GRAPose(lat=-35.363328, lon=149.1652241, alt=5.0, heading=90.0),
  GRAPose(lat=-35.36332799995002, lon=149.16511394398992, alt=5.0, heading=90.0),
  GRAPose(lat=-35.36323816790049, lon=149.16511394411248, alt=5.0, heading=90.0),
  GRAPose(lat=-35.36323816795048, lon=149.1652241, alt=5.0, heading=90.0),
  GRAPose(lat=-35.363328, lon=149.1652241, alt=5.0, heading=90.0)],
 [GRAPose(lat=-35.36323816795048, lon=149.1652241, alt=5.0, heading=90),
  GRAPose(lat=-35.36323816790049, lon=149.16511394411248, alt=5.0, heading=90),
  GRAPose(lat=-35.36314833585095, lon=149.16511394423503, alt=5.0, heading=90),
  GRAPose(lat=-35.36314833590094, lon=149.1652241, alt=5.0, heading=90),
  GRAPose(lat=-35.36323816795048, lon=149.1652241, alt=5.0, heading=90)],
 [GRAPose(lat=-35.36337291597479, lon=149.16533425607133, alt=5.0, heading=90),
  GRAPose(lat=-35.36337291602477, lon=149.1652241, alt=5.0, heading=90),
  GRAPose(lat=-35.36328308397523, lon=149.1652241, alt=5.0, heading=90),
  GRAPose(lat=-35.36

## Save the missions

In [5]:
for i,gra_path in enumerate(gra_paths):
    save_mission(name=mission_name+f"_{i+1}",poses=gra_path)

## Visualization Parameters

In [6]:

markers = [
	ConfigGazebo.create_markertraj(
		traj=enu_path,
		color=color
	)
	for enu_path, color in zip(enu_paths, colors)
]
Gazebo.plot_3d_interactive(
	markers=markers,
	title="Simulation Waypoints",
	frames=(0.2, 0.2, 0.6),
	ground=-0.05
)

## Visualizer Config

In [7]:
novis=NoneVisualizer()


gaz_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
                           models=list(zip(n_uavs*["iris"],colors)),
                           marker_trajs = markers)
gaz= Gazebo(gra_origin,enu_homes,gaz_config)

qgc=QGC(gra_homes)

## Launch Simulatotion

In [8]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[gaz],
	n_uavs=n_uavs,
	terminals=False,
	verbose=1,
)
orac = simulator.launch(gcs_sysids={'GCS': [1]})

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 3437865)
🚀 ArduPilot SITL vehicle 3 launched (PID 3437869)
🚀 ArduPilot SITL vehicle 2 launched (PID 3437867)
🚀 ArduPilot SITL vehicle 4 launched (PID 3437871)
🚀 ArduPilot SITL vehicle 5 launched (PID 3437873)
🚀 UAV logic for vehicle 1 launched (PID 3437875)
🚀 UAV logic for vehicle 2 launched (PID 3437878)
🚀 UAV logic for vehicle 3 launched (PID 3437876)
🚀 UAV logic for vehicle 4 launched (PID 3437877)
🚀 UAV logic for vehicle 5 launched (PID 3437879)
🚀 Proxy for vehicle 1 launched (PID 3437884)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
🚀 Proxy for vehicle 2 launched (PID 3437889)
🔗 UAV logic 2 is connected to Ardupilot SITL vehicle 2
🚀 Proxy for vehicle 4 launched (PID 3437893)
🔗 UAV logic 4 is connected to Ardupilot SITL vehicle 4
🚀 Proxy for vehicle 3 launched (PID 3437892)
🔗 UAV logic 3 is connected to Ardupilot SITL vehicle 3
🚀 Proxy for vehicle 5 launched (PID 343

bind port 5800 for SERIAL0
SERIAL0 on TCP port 5800
bind port 5830 for SERIAL0
SERIAL0 on TCP port 5830
bind port 5810 for SERIAL0
SERIAL0 on TCP port 5810
bind port 5820 for SERIAL0
SERIAL0 on TCP port 5820
bind port 5790 for SERIAL0
SERIAL0 on TCP port 5790


Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=3
Starting SITL Gazebo
Home: -35.363328 149.165224 alt=0.000000m hdg=90.000000
Bind 127.0.0.1:9023 for SITL in
Setting Gazebo interface to 127.0.0.1:9022 
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9025
Waiting for connection ....
SIM_VEHICLE: Start
SIM_VEHICLE: Starting up at [-35.363328, 149.1652241, 0.0, 90.0]
SIM_VEHICLE: Using defaults from (../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=4
Starting SITL Gazebo
Home: -35.363328 149.165224 alt=0.000000m hdg=90.000000
Bind 127.0.0.1:9033 for SITL in
Setting Gazebo interface to 127.0.0.1:9032 
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9035
Waiting for connection ....
SIM_VEHICLE: Starting up at [-35.363328, 149.1652241, 0.0, 90

Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.


Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/def

bind port 5812 for SERIAL1
SERIAL1 on TCP port 5812
bind port 5813 for SERIAL2
SERIAL2 on TCP port 5813
bind port 5822 for SERIAL1
SERIAL1 on TCP port 5822
bind port 5823 for SERIAL2
SERIAL2 on TCP port 5823
bind port 5832 for SERIAL1
SERIAL1 on TCP port 5832
bind port 5833 for SERIAL2
SERIAL2 on TCP port 5833
bind port 5792 for SERIAL1
SERIAL1 on TCP port 5792
bind port 5793 for SERIAL2
SERIAL2 on TCP port 5793
bind port 5802 for SERIAL1
SERIAL1 on TCP port 5802
bind port 5803 for SERIAL2
SERIAL2 on TCP port 5803
validate_structures:528: Validating structures
validate_structures:528: Validating structures
validate_structures:528: Validating structures
validate_structures:528: Validating structures
validate_structures:528: Validating structures


Vehicle 2: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 2: ▶️ Step Started: clear uav missions


## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Step Started: clear uav missions


🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
Vehicle 4: ✅ Step Done: clear uav missions
Vehicle 2: ✅ Step Done: clear uav missions
Vehicle 3: ✅ Step Done: clear uav missions
Vehicle 5: ✅ Step Done: clear uav missions
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 4: ✅ 7 waypoints read.
Vehicle 2: ✅ 7 waypoints read.
Vehicle 3: ✅ 7 waypoints read.
Vehicle 5: ✅ 7 waypoints read.
Vehicle 1: ✅ 7 waypoints read.
Vehicle 4: ✅ Sent mission item 0
Vehicle 2: ✅ Sent mission item 0
Vehicle 3: ✅ Sent mission item 0
Vehicle 5: ✅ Sent mission item 0
Vehicle 1: ✅ Sent mission item 0
Vehicle 4: ✅ Sent mission item 1
Vehicle 2: ✅ Sent mission item 1
Vehicle 3: ✅ Sent mission item 1
Vehicle 5: ✅ Sent mission item 1
Vehicle 1: ✅ Sent mission item 1
Vehicle 4: ✅ Sent mission item 2
Vehicle 2: ✅ Sent mission item 2
Vehicle 3: ✅ Sent